# <font color= F30720> <b> <i> Modelando la serie de la temperatura </i> </b> </font>
La serie de la temperatura fue extraída de la base de datos del IDEAM, se tuvieron en cuenta los promedios diarios de las temperaturas en grados centigrados (°C) registradas en Bogotá en las diferentes estaciones meteorológicas que recolectaron información de esos días.
La serie de tiempo cuenta con un total de 1826 registros, de los cuales 13 (0.7%) fueron imputados puesto que no se presentaba la información necesaria. Esta imputación fue realizada a partir del método de vecinos más cercanos (KNN), donde se tuvierón en cuenta 5 vecinos.

Para el modelar la serie de la temperatura se usara inicialmente un modelo SARIMA, seguido de redes recurrentes simples, LSTM y finalmente LSTM.
Para seleccionar el mejor modelo se tomará como criterio el error cuadrático medio.

## <font color= 199EDC> <b> Red Neuronal Recurrente LSTM </b> </font>

#### <font color= blue>  Importación de Datos</font>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

import sklearn as sk
from sklearn import impute
from sklearn import preprocessing
import sklearn.externals
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.impute import KNNImputer
import sklearn.preprocessing

from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import time
import sklearn.externals
import joblib
import plotly.graph_objects as go
from sklearn import metrics

import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.statespace.sarimax import SARIMAX

import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Mineria /Temperatura1.csv", sep=';', header=0, decimal = ',')
Fecha = pd.date_range(start='2017-01-01', end='2021-12-31', freq='D')
df['Fecha'] = Fecha
df = df.set_index('Fecha')

print(df[pd.isnull(df.ValorObservado)])
print('En total hay' ,
      str(df['ValorObservado'].isnull().sum()) ,
      'valores sin información')
print('Correspondientes al {:.3f}% del total'
      .format(df['ValorObservado'].isnull().sum()*100/len(df)))

            ValorObservado
Fecha                     
2017-08-12             NaN
2017-12-24             NaN
2019-09-15             NaN
2019-09-16             NaN
2019-09-17             NaN
2020-11-12             NaN
2021-01-05             NaN
2021-01-06             NaN
2021-01-07             NaN
2021-01-08             NaN
2021-08-18             NaN
2021-08-20             NaN
2021-12-05             NaN
En total hay 13 valores sin información
Correspondientes al 0.712% del total


La serie presenta valores faltantes, por lo tantol se imputaran usando el método de vecinos más cercanos (KNN), como se muestra a continuación.

#### <font color= blue> Imputación a partir del vecino más cercano</font>

In [ ]:
#Imputación de Valores usando el vecino más cercano
imput = KNNImputer(n_neighbors=5, weights="uniform")

# Ajustamos el modelo e imputamos los missing values
imput.fit(df[['ValorObservado']])
df['ValorObservado'] = imput.transform(df[['ValorObservado']]).ravel()
print()
print("Valores pérdidos en ValorObservado: " , 
      str(df['ValorObservado'].isnull().sum()))


Valores pérdidos en ValorObservado:  0


In [ ]:
fig = px.line(df, x=df.index, y="ValorObservado")
fig.update_xaxes(title_text="Fecha")
fig.show()

#### <font color= blue> Separación de datos entrenamiento y validación  </font>
Para el respectivo análisis se tomarán el 80% de los datos para entrenamiento y validacion, el 20% restantes para prueba, dichos valores corresponden a 1460 y 365 respectivamente.

In [ ]:
# split data in 80%/20% train/test sets
test_set_size_percentage = 20

min_max_scaler = sklearn.preprocessing.MinMaxScaler()

# function for min-max normalization of stock
def normalize_data(df):
    df["ValorObservado"] = min_max_scaler.fit_transform(
        df.ValorObservado.values.reshape(-1, 1)
    )
    return df


# function to create train, test data given stock data and sequence length
def load_data(df, seq_len):
    data_raw = df.values  # convert to numpy array
    data = []

    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len):
        data.append(data_raw[index : index + seq_len])

    data = np.array(data)
    test_set_size = int(np.round(test_set_size_percentage / 100 * data.shape[0]))
    train_set_size = data.shape[0] - test_set_size

    X_train = data[:train_set_size, :-1, :]
    y_train = data[:train_set_size, -1, :]

    X_test = data[train_set_size:, :-1, :]
    y_test = data[train_set_size:, -1, :]

    return [X_train, y_train, X_test, y_test]


cols = list(df.columns.values)
print("df.columns.values = ", cols)

# normalize stock
df_norm = df.copy()
df_norm = normalize_data(df_norm)

# create train, test data
seq_len = 20  # choose sequence length
X_train, y_train, X_test, y_test = load_data(df_norm, seq_len)
print("X_train.shape = ", X_train.shape)
print("y_train.shape = ", y_train.shape)
print("X_test.shape = ", X_test.shape)
print("y_test.shape = ", y_test.shape)


df.columns.values =  ['ValorObservado']
X_train.shape =  (1445, 19, 1)
y_train.shape =  (1445, 1)
X_test.shape =  (361, 19, 1)
y_test.shape =  (361, 1)


In [ ]:
fig = px.line(df_norm, x=df.index, y="ValorObservado")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservadoNormalizado")
fig.show()

#### <font color= blue> Modelo </font>

In [ ]:
# SRNN architecture
LSTM_model = Sequential()
# First LSTM layer with Dropout regularisation
LSTM_model.add(L.LSTM(units=50, input_shape=(seq_len - 1, 1), return_sequences=True))
LSTM_model.add(L.Dropout(0.2))
# Second LSTM layer
LSTM_model.add(L.LSTM(units=50, return_sequences=True))
LSTM_model.add(L.Dropout(0.2))
# Third LSTM layer
LSTM_model.add(L.LSTM(units=50, return_sequences=True))
LSTM_model.add(L.Dropout(0.2))
# Fourth LSTM layer
LSTM_model.add(L.LSTM(units=50))
LSTM_model.add(L.Dropout(0.5))
# The output layer
LSTM_model.add(L.Dense(units=50, kernel_initializer="uniform", activation="tanh"))
LSTM_model.add(L.Dense(units=1, kernel_initializer="uniform", activation="linear"))

earlystop = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=50)
callbacks_list = [earlystop]

# Compiling the RNN
LSTM_model.compile(optimizer="adam", loss="mean_squared_error")
# Fitting to the training set
start = time.time()
LSTM = LSTM_model.fit(
     X_train,
     y_train,
     epochs=5,
     batch_size=35,
     validation_split=0.05,
     verbose=1,
     callbacks=callbacks_list,
)
print("compilation time : ", time.time() - start)

Epoch 1/5
40/40 [==============================] - 19s 121ms/step - loss: 0.0838 - val_loss: 0.0230
Epoch 2/5
40/40 [==============================] - 2s 46ms/step - loss: 0.0288 - val_loss: 0.0137
Epoch 3/5
40/40 [==============================] - 2s 46ms/step - loss: 0.0202 - val_loss: 0.0163
Epoch 4/5
40/40 [==============================] - 2s 45ms/step - loss: 0.0164 - val_loss: 0.0149
Epoch 5/5
40/40 [==============================] - 2s 45ms/step - loss: 0.0156 - val_loss: 0.0152
compilation time :  26.31064248085022


In [ ]:
joblib.dump(LSTM_model,'/content/drive/Shareddrives/Mineria /LSTM')

['/content/drive/Shareddrives/Mineria /LSTM']

In [ ]:
LSTM_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 19, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 19, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 19, 50)            20200     
                                                                 
 dropout_1 (Dropout)         (None, 19, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 19, 50)            20200     
                                                                 
 dropout_2 (Dropout)         (None, 19, 50)            0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                2

In [ ]:
LSTM_losses = pd.DataFrame(LSTM.history)
fig = px.line(LSTM_losses, x=LSTM_losses.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

#### <font color= blue> Predicciones (1 paso adelante)</font>

In [ ]:
LSTM_Predict = LSTM_model.predict(X_test)
LSTM_Predict = min_max_scaler.inverse_transform(LSTM_Predict)

12/12 [==============================] - 2s 12ms/step


In [ ]:
# plot y_train, y_test, and testPredict using plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train) + seq_len],
        y=min_max_scaler.inverse_transform(y_train).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=min_max_scaler.inverse_transform(y_test).ravel(),
        mode="lines",
        name="Prueba",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=LSTM_Predict.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

#### <font color= blue>  Intervalos de Confianza (1 paso adelante) </font>

In [ ]:
def QuantileLoss(perc, delta=1e-4):
    perc = np.array(perc).reshape(-1)
    perc.sort()
    perc = perc.reshape(1, -1)
    def _qloss(y, pred):
        I = tf.cast(y <= pred, tf.float32)
        d = K.abs(y - pred)
        correction = I * (1 - perc) + (1 - I) * perc
        # huber loss
        huber_loss = K.sum(correction * tf.where(d <= delta, 0.5 * d ** 2 / delta, d - 0.5 * delta), -1)
        # order loss
        q_order_loss = K.sum(K.maximum(0.0, pred[:, :-1] - pred[:, 1:] + 1e-6), -1)
        return huber_loss + q_order_loss
    return _qloss

In [ ]:
# quantiles
perc_points = [0.05, 0.95]

# SRNN architecture
qLSTM_model = Sequential()
# First GRU layer with Dropout regularisation
qLSTM_model.add(L.LSTM(units=50, input_shape=(seq_len - 1, 1)))
qLSTM_model.add(L.Dropout(0.2))

# The output layer
qLSTM_model.add(L.Dense(units=50, kernel_initializer="uniform", activation="tanh"))
qLSTM_model.add(L.Dense(units=len(perc_points), kernel_initializer="uniform", activation="linear"))

earlystop = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=50)
callbacks_list = [earlystop]

# Compiling the RNN
qLSTM_model.compile(optimizer="adam", loss=QuantileLoss(perc_points))
# Fitting to the training set
start = time.time()
qLSTM = qLSTM_model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=35,
    validation_split=0.05,
    verbose=1,
    callbacks=callbacks_list,
)
print("compilation time : ", time.time() - start)

Epoch 1/100
40/40 [==============================] - 4s 28ms/step - loss: 0.2284 - val_loss: 0.0516
Epoch 2/100
40/40 [==============================] - 0s 12ms/step - loss: 0.0344 - val_loss: 0.0274
Epoch 3/100
40/40 [==============================] - 0s 12ms/step - loss: 0.0273 - val_loss: 0.0275
Epoch 4/100
40/40 [==============================] - 1s 13ms/step - loss: 0.0251 - val_loss: 0.0294
Epoch 5/100
40/40 [==============================] - 1s 13ms/step - loss: 0.0247 - val_loss: 0.0269
Epoch 6/100
40/40 [==============================] - 1s 13ms/step - loss: 0.0229 - val_loss: 0.0277
Epoch 7/100
40/40 [==============================] - 1s 13ms/step - loss: 0.0232 - val_loss: 0.0273
Epoch 8/100
40/40 [==============================] - 1s 20ms/step - loss: 0.0228 - val_loss: 0.0257
Epoch 9/100
40/40 [==============================] - 1s 19ms/step - loss: 0.0222 - val_loss: 0.0268
Epoch 10/100
40/40 [==============================] - 1s 19ms/step - loss: 0.0227 - val_loss: 0.0255

In [ ]:
qLSTM_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_47 (LSTM)              (None, 50)                10400     
                                                                 
 dropout_43 (Dropout)        (None, 50)                0         
                                                                 
 dense_29 (Dense)            (None, 50)                2550      
                                                                 
 dense_30 (Dense)            (None, 2)                 102       
                                                                 
Total params: 13,052
Trainable params: 13,052
Non-trainable params: 0
_________________________________________________________________


In [ ]:
qLSTM_losses = pd.DataFrame(qLSTM.history)
fig = px.line(qLSTM_losses, x=qLSTM_losses.index, y=["loss", "val_loss"])
fig.update_xaxes(title_text="Epoch")
fig.update_yaxes(title_text="Loss")
fig.show()

In [ ]:
qLSTM_Predict = qLSTM_model.predict(X_test)
qLSTM_Predict = min_max_scaler.inverse_transform(qLSTM_Predict)

12/12 [==============================] - 1s 4ms/step


In [ ]:
qLSTM_Predict

array([[12.938789 , 14.692345 ],
       [11.116115 , 13.247358 ],
       [10.996713 , 13.098039 ],
       [10.94361  , 13.009496 ],
       [10.9192095, 12.953624 ],
       [10.90643  , 12.915748 ],
       [12.126468 , 13.946181 ],
       [12.446295 , 14.292166 ],
       [12.913088 , 14.764404 ],
       [13.080907 , 14.991259 ],
       [12.956506 , 14.952547 ],
       [12.922914 , 14.957763 ],
       [13.152697 , 15.171972 ],
       [12.830343 , 14.925642 ],
       [13.031329 , 15.090541 ],
       [13.357614 , 15.377728 ],
       [12.895368 , 15.0149765],
       [12.494859 , 14.658253 ],
       [12.790086 , 14.870004 ],
       [12.782907 , 14.858242 ],
       [13.147524 , 15.1648035],
       [12.832102 , 14.920617 ],
       [12.527753 , 14.6535845],
       [12.510369 , 14.611149 ],
       [12.604079 , 14.671206 ],
       [12.659135 , 14.71186  ],
       [13.538659 , 15.460687 ],
       [12.566691 , 14.699536 ],
       [12.233394 , 14.386821 ],
       [12.930529 , 14.938287 ],
       [13

In [ ]:
# plot y_train, y_test, and testPredict using plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df.index[seq_len : len(y_train) + seq_len],
        y=min_max_scaler.inverse_transform(y_train).ravel(),
        mode="lines",
        name="Entrenamiento",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=LSTM_Predict.ravel(),
        mode="lines",
        name="Predicción",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=qLSTM_Predict[:,0] ,
        mode="lines",
        name="0.025",
    )
)
fig.add_trace(
    go.Scatter(
        x=df.index[len(y_train) + seq_len :],
        y=qLSTM_Predict[:,1],
        mode="lines",
        name="0.975",
    )
)
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="ValorObservado")
fig.show()

#### <font color= blue>  Error Cuadrático Medio (1 paso adelante) </font>

In [ ]:
LSTM_Score = metrics.mean_squared_error(y_test, LSTM_Predict) ** .5
print('Test Score: %.2f RMSE' % (LSTM_Score))

Test Score: 11.35 RMSE
